In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn import preprocessing # LabelEncoder
from sklearn.preprocessing import MinMaxScaler # Escala los datos
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import scikitplot as skplt 
from string import ascii_uppercase 
# import seaborn as sns
import qgrid
import time

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.ensemble import AdaBoostClassifier
from scipy import stats #Para la moda
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import ShuffleSplit

In [3]:
df=pd.read_csv('CSV ready to model/withoutPCA.csv')

In [4]:
le=LabelEncoder()
df['GRAVEDAD']=df.loc[:,['GRAVEDAD']].apply(le.fit_transform)
y=df['GRAVEDAD']
x=df.drop(['GRAVEDAD','PERIODO'],axis=1)

In [5]:
def select_features(modelo, n_features, fwd, fltg):

    sfs = SFS(modelo, 
           k_features=n_features,
           forward=fwd,
           floating=fltg,
           verbose=0,
           scoring='accuracy',
           cv=0,
           n_jobs=-1)
    
    return sfs

In [6]:

def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

In [7]:
def selection(X,Y,features):
    print(features)
    fwd = True
    fltg = True
        
    #Para calcular el costo computacional
    tiempo_i = time.time()

    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,3])
    recall_list = np.zeros([4,3])
    f_list = np.zeros([4,3])
    
    feature_list = [] #Para guardar los indices de las características que eligió en cada iteración
    XN_list = [] #Para guardar los diferentes X
    sf_list = []
    #model = RandomForestClassifier(n_estimators=100, max_features =2, n_jobs = -1)
    #Implemetamos la metodología de validación 
#     model = svm.SVC(decision_function_shape='ovo', kernel='rbf', C = 10, gamma = 1,n_jobs=-1)
    model = AdaBoostClassifier(n_estimators=50)

    Errores = np.ones(4)
    
    for j in range(4):

        Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.25) # Modificar metodología de validación
        sf = select_features(model, features, fwd, fltg)
        sf =  sf.fit(Xtrain, Ytrain)

        X_train_sfs = sf.transform(Xtrain)
        X_test_sfs = sf.transform(Xtest)
        XN = sf.transform(X)

        model.fit(X_train_sfs, Ytrain)
        pred = model.predict(X_test_sfs)
        
        #code for calculating accuracy 
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall 
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision 
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_
        
        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_

        Errores[j] = classification_error(pred, Ytest)
        
        feature_list.extend(sf.k_feature_idx_)
        
        XN_list.append(XN)
        sf_list.append(sf)
    
    
    index = np.where(accuracy_list == np.amax(accuracy_list)) #Busca cual es el resultado más grande en la lista de accuracy(eficiencia)   
    idx = index[0][0] #pasa el índice de array a entero   
    XN = XN_list[idx] #Selecciona el mejor X 
    sf = sf_list[idx] #índices de las mejores características
    
    result={'xn':XN,'features':sf,'Eficiencia':np.mean(accuracy_list),'Int_Eficiencia':np.std(accuracy_list),
    'Sensibilidad':np.mean(recall_list),'Int_Sensibilidad':np.std(recall_list),
    'Precisión':np.mean(precision_list),'Int Precisión':np.std(precision_list),
    'F':np.mean(f_list),'Int_F':np.std(f_list),
    'Error':np.mean(Errores),'Int_Error':np.std(Errores),
    'Tiempo':time.time()-tiempo_i
    }
    return  result


In [8]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({'# de características seleccionadas' : pd.Series([2,3,4,5,6,7,8,9])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""
df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['# de características seleccionadas'], inplace=True)

for k in df_types.index:
    result= selection(x,y,k)
    print(k)
    print(result['features'].k_feature_idx_)

    df_types["Eficiencia"][k] = result['Eficiencia']
    df_types["Int_Eficiencia"][k] = result['Int_Eficiencia']
    df_types["Sensibilidad"][k] = result['Sensibilidad']
    df_types["Int_Sensibilidad"][k] = result['Int_Sensibilidad']
    df_types["Precision"][k] = result['Precisión']
    df_types["Int_Precision"][k] = result['Precisión']
    df_types["F-Score"][k] = result['F']
    df_types["Int_F-Score"][k] = result['Int_F']
    df_types["Error_Prueba"][k] = result['Error']
    df_types["Int_error"][k] = result['Int_Error']
    df_types["Tiempo de ejecución"][k] = result['Tiempo']
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)

2
2
(1, 4)
3
3
(1, 4, 6)
4
4
(1, 3, 4, 6)
5
5
(1, 2, 3, 4, 6)
6
6
(0, 2, 3, 5, 6, 8)
7
7
(0, 2, 3, 4, 5, 6, 7)
8
8
(0, 1, 2, 3, 4, 6, 7, 8)
9
9
(0, 1, 2, 3, 4, 5, 6, 7, 8)


In [9]:
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [ ]:
1)standar scaler 8 caracteristicas
2)min max 8 caracteristicas
3)standar scaler 9 caracteristicas
4)min max 9 caracteristicas